## Import modules

In [1]:
import cobra
from tqdm import tqdm
import numpy as np
import math

## Test Model

In [2]:
import cobra.test
model = cobra.test.create_test_model('textbook')
model

Name,e_coli_core
Memory address,0x0112d05e48
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"


## Model

In [3]:
# model = cobra.io.read_sbml_model('Models/iAF1260.xml')
# model

## Elimination list and model config

In [4]:
elilist = model.exchanges
model.solver = 'glpk'

## Core model optimization

In [5]:
solWT = model.optimize()
# solWT

## Objective value and fluxes

In [6]:
grWT = solWT.objective_value
J = solWT.fluxes

## Flux operations to obtain Jnz

In [7]:
Jnz_before_filtering = np.flatnonzero(J)
eliIdx = [model.reactions.index(reaction_id) for reaction_id in elilist]
Jnz = np.setdiff1d(Jnz_before_filtering,eliIdx)
# Jnz

## Single lethal reactions

In [8]:
Jsl_idx = []
for delIdx_i in Jnz:
    with model:
        model.reactions[delIdx_i].knock_out()
        solKO_i = model.slim_optimize()
        if solKO_i < 0.01 * grWT or math.isnan(solKO_i) == True:
            Jsl_idx.append(int(delIdx_i))
Jsl_idx = np.array(Jsl_idx)
# Jsl_idx

In [10]:
Jsl_idx.shape

(14,)

## Eliminating single lethal reactions to continue further

In [11]:
Jnz_copy = np.setdiff1d(Jnz,Jsl_idx)
# Jnz_copy

## Double and Triple lethal reactions

### First part

In [12]:
Jdl_idx = []
Jtl_idx = []

for delIdx_i in tqdm(Jnz_copy,desc='Part 1 of 2'):
    with model:
        model.reactions[delIdx_i].knock_out()
        solKO_i = model.optimize()
        newnnz = np.flatnonzero(solKO_i.fluxes)
        Jnz_i_before_filtering = np.setdiff1d(newnnz,Jnz)
        Jnz_i = np.setdiff1d(Jnz_i_before_filtering,eliIdx)

        for delIdx_j in Jnz_i:
            with model:
                model.reactions[delIdx_j].knock_out()
                solKO_ij = model.slim_optimize()
                if solKO_ij < 0.01 * grWT and math.isnan(solKO_ij) is False:
                    Jdl_idx.append([int(delIdx_i),int(delIdx_j)])
                    
                elif math.isnan(solKO_ij) is True:
                    solKO_ij_infeasibility = model.optimize()
                    if solKO_ij_infeasibility.objective_value < 0.01 * grWT or math.isnan(solKO_ij_infeasibility.objective_value) == True:
                        Jdl_idx.append([int(delIdx_i),int(delIdx_j)])
#                         continue

                    Jnz_ij_before_filtering = np.flatnonzero(solKO_ij_infeasibility.fluxes)
                    Jnz_ij_after_filtering = np.setdiff1d(Jnz_ij_before_filtering,Jnz_before_filtering)
                    Jnz_ij = np.setdiff1d(Jnz_ij_after_filtering,eliIdx)
                    
                    for delIdx_k in Jnz_ij:
                        with model:
                            model.reactions[delIdx_k].knock_out()
                            solKO_ijk = model.slim_optimize()
                            if solKO_ijk < 0.01 * grWT or math.isnan(solKO_ijk) == True:
                                Jtl_idx.append([int(delIdx_i),int(delIdx_j),int(delIdx_k)])

Part 1 of 2: 100%|██████████| 28/28 [00:00<00:00, 143.61it/s]


In [13]:
len(Jdl_idx)

16

In [14]:
len(Jtl_idx)

19

### Second part

In [15]:
for delIdx_i in tqdm(Jnz_copy,desc='Part 2 of 2'):
    for delIdx_j in Jnz_copy:
        if np.where(Jnz_copy==delIdx_j) < np.where(Jnz_copy==delIdx_i):
            with model:
                model.reactions[delIdx_i].knock_out()
                model.reactions[delIdx_j].knock_out()
                solKO_ij = model.slim_optimize()
                if solKO_ij < 0.01 * grWT and math.isnan(solKO_ij) is False:
                    Jdl_idx.append([int(delIdx_i),int(delIdx_j)])
                
                elif math.isnan(solKO_ij) is True:
                    solKO_ij = model.optimize()
                    if solKO_ij.objective_value < 0.01 * grWT or math.isnan(solKO_ij.objective_value) == True:
                        Jdl_idx.append([int(delIdx_i),int(delIdx_j)])
#                          continue

                    Jnz_ij_before_filtering = np.flatnonzero(solKO_ij.fluxes)
                    Jnz_ij_after_filtering = np.setdiff1d(Jnz_ij_before_filtering,Jnz_before_filtering)
                    Jnz_ij = np.setdiff1d(Jnz_ij_after_filtering,eliIdx)

                    for delIdx_k in Jnz_ij:
                        with model:
                            solKO_ijk = model.slim_optimize()
                            if solKO_ijk < 0.01 * grWT or math.isnan(solKO_ijk) == True:
                                Jtl_idx.append([int(delIdx_i),int(delIdx_j),int(delIdx_k)])

                    for delIdx_k in Jnz_copy:
                        with model:
                            if np.where(Jnz_copy==delIdx_k) < np.where(Jnz_copy==delIdx_j):
                                solKO_ijk = model.slim_optimize()
                                if solKO_ijk < 0.01 * grWT or math.isnan(solKO_ijk) == True:
                                    Jtl_idx.append([int(delIdx_i),int(delIdx_j),int(delIdx_k)])

# Eliminate double lethal reaction deletions in triple lethal reacutions
Jdl_idx = np.array(Jdl_idx)
Jtl_idx = np.array(Jtl_idx)

Part 2 of 2: 100%|██████████| 28/28 [00:01<00:00, 21.49it/s]


In [16]:
Jdl_idx.shape

(88, 2)

In [17]:
Jtl_idx.shape

(906, 3)

In [18]:
temporary = []
g = np.zeros(Jdl_idx.shape[0])
for delIdx_i in Jtl_idx:
    for delIdx_j in Jdl_idx:
        g[np.where(Jdl_idx==delIdx_j)[0][0]] = np.sum(np.in1d(delIdx_i,delIdx_j))
        if g[np.where(Jdl_idx==delIdx_j)[0][0]] >= 2:
            break
#             print(g[np.where(Jdl_idx==delIdx_j)[0][0]])
    if np.max(g) < 2:
#         print(g[np.where(Jdl_idx==delIdx_j)[0][0]])
        temporary.append(delIdx_i)

# Jtl_idx_copy = np.array(temporary)
# Jtl_idx_copy = np.unique(np.sort(Jtl_idx_copy), axis=0)

#     Jdl = [model.reactions.get_by_any(rxn_pair_idx) for rxn_pair_idx in Jdl_idx]
#     Jtl = [model.reactions.get_by_any(rxn_triplet_idx) for rxn_triplet_idx in Jtl_idx]

In [19]:
temporary

[]